In [21]:
import kagglehub


path = kagglehub.dataset_download("mssmartypants/rice-type-classification")

print("Path to dataset files:", path)

import os
path = "/Users/gauravkumar/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2"
print(os.listdir(path))


Path to dataset files: /Users/gauravkumar/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2
['riceClassification.csv']


In [14]:
import torch
import torch.nn as nn
from torch.optim import adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [28]:
data_df = pd.read_csv("/Users/gauravkumar/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2/riceClassification.csv")

In [29]:
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [31]:
data_df.dropna(inplace=True)
data_df.drop(['id'], axis=1, inplace=True)

In [32]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [34]:
print(data_df["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [38]:
orginal_df = data_df.copy()

for columns in data_df.columns: #normalisation
    data_df[columns]= data_df[columns]/data_df[columns].abs().max()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [45]:
X = np.array(data_df.iloc[:,:-1])
y = np.array(data_df.iloc[:,-1])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

In [50]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [51]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12729, 10) (1364, 10) (12729,) (1364,)


In [54]:
    class Dataset(Dataset):
        def __init__(self, X, y):
            self.X = torch.tensor(X, dtype=torch.float32)
            self.y = torch.tensor(y, dtype=torch.float32)
        def __len__(self):
            return len(self.X)
        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]


In [55]:
training_set = Dataset(X_train, y_train)
validation_set = Dataset(X_val, y_val)
test_set = Dataset(X_test, y_test)
